In [1]:
!pip install torch datasets accelerate peft  transformers==4.31.0 trl
!pip install bitsandbytes==0.40.0.post4
!pip install scipy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://mirrors.aliyun.com/pypi/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://mirrors.aliyun.com/pypi/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://mirrors.aliyun.com/pypi/simple


In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = "0"


import warnings
warnings.filterwarnings("ignore") #Python忽略代码运行时产生的警告



/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
base_model = "Llama2-7b-chat-hf"       # 基础模型路径
guanaco_dataset = "guanaco-llama2-1k"  # 数据集路径
new_model = "llama-2-7b-chat-guanaco"  # 微调模型名称
dataset = load_dataset(guanaco_dataset, split="train") # 加载微调数据集

In [4]:

compute_dtype = getattr(torch, "float16") # 用于线性层计算的数据类型             
quant_config = BitsAndBytesConfig(      # 量化参数
    load_in_4bit=True,                  # 启用 4 位加载
    bnb_4bit_quant_type="nf4",          # 指定用于量化的数据类型nf4 。
    bnb_4bit_compute_dtype=torch.bfloat16,# 用于线性层计算的数据类型
    bnb_4bit_use_double_quant=False,    # 是否使用嵌套量化来提高内存效率
)


In [5]:
model = AutoModelForCausalLM.from_pretrained(  # 加载基础模型
    base_model,
    quantization_config=quant_config,
    #device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1    # 设置张量并行


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /root/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so...


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.47s/it]


In [6]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True) # 加载tokenizer
tokenizer.pad_token = tokenizer.eos_token    # 序列结束的标记eos_token默认是 [SEP]
tokenizer.padding_side = "right"             # padding_side 设置为right以修复 fp16 的问题


In [7]:

peft_params = LoraConfig(   # 指定LoRA微调参数
    lora_alpha=16,          # LoRA 缩放因子 
    lora_dropout=0.1,       # dropout的比例
    r=64,                   # 更新矩阵的秩
    bias="none",            # 指定是否应训练 bias 参数。
    task_type="CAUSAL_LM",  # 模型任务类型，这里是CAUSAL_LM任务
)

training_params = TrainingArguments(  # 训练器参数
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    #report_to="tensorboard"
)

In [ ]:

trainer = SFTTrainer(         # 使用SFTTrainer初始化训练器
    model=model,              # 指定要训练的模型的路径
    train_dataset=dataset,    # 指定用于训练的数据集路径
    peft_config=peft_params,  # 指定数据集中用于训练的文本字段
    dataset_text_field="text",# 指定数据集中包含文本的字段名称
    max_seq_length=None,      # 定义序列的最大长度限制
    tokenizer=tokenizer,      # 定义将文本数据转换为模型输入的tokenizer
    args=training_params,     # 定义训练过程中的其他设置和参数
    packing=False,            # 不将多个权重参数打包成更少的数据单元进行存储和传输
)

trainer.train()               # 使用trainer训练模型

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.347900
50,1.621100
75,1.221500
100,1.449700
125,1.188900
150,1.374500
175,1.182300
200,1.472300


In [9]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('llama-2-7b-chat-guanaco/tokenizer_config.json',
 'llama-2-7b-chat-guanaco/special_tokens_map.json',
 'llama-2-7b-chat-guanaco/tokenizer.json')

In [10]:
#在FP16中重新加载模型并将其与LoRA权重合并
load_model = AutoModelForCausalLM.from_pretrained( 
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

model = PeftModel.from_pretrained(load_model, new_model)
model = model.merge_and_unload()

#重新加载tokenizer以保存它
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


model.save_pretrained(new_model)    #保存模型到本地
tokenizer.save_pretrained(new_model)#保存tokenizer到本地

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.23s/it]


('llama-2-7b-chat-guanaco/tokenizer_config.json',
 'llama-2-7b-chat-guanaco/special_tokens_map.json',
 'llama-2-7b-chat-guanaco/tokenizer.json')

In [11]:
prompt = "Who is Leonardo Da Vinci?"   #使用新模型进行文本生成
pipe = pipeline(task="text-generation", model=new_model, tokenizer=new_model, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.79s/it]
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


<s>[INST] Who is Leonardo Da Vinci? [/INST] Leonardo da Vinci was a famous Italian artist, inventor, engineer, and scientist.Љ Leonardo was born in 1452 in the village of Vinci in the Tuscany region of Italy. He began his career as an artist in the studio of Andrea del Verrocchio, where he trained in painting, sculpture, and engineering. He later became an independent artist and inventor, and his works include the famous painting "Mona Lisa" and many other inventions and designs. Leonardo da Vinci is considered one of the greatest artists of all time, and his legacy continues to inspire people around the world.  [INST] How did he die? [/INST] Leonardo da Vinci died on May 2, 1519, at the age of 67. He was buried in the Chapel of Saint-Hubert in


In [12]:
prompt = "What is Datacamp Career track?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is Datacamp Career track? [/INST] DataCamp Career Track is a program that offers a structured learning path and career support to help you land your dream job in data science. Earn a verified certificate and get career support from our team. 

DataCamp Career Track is designed to help you master the skills you need to succeed in data science, with a focus on practical applications and real-world projects. Our curriculum is built around the skills that are most in demand in the industry, and we offer a range of courses and specializations to help you focus on the areas that interest you most. 

With DataCamp Career Track, you'll have access to a range of resources and support to help you succeed in your career. Our team of career coaches will work with you one-on-one to help you identify your strengths and weaknesses, set career goals, and develop a plan to achieve


In [14]:
#python查看包版本并输出到txt
!pip freeze > requirements.txt


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
# Push model to the hf hub

#model.push_to_hub(new_model, use_temp_dir=False)
#tokenizer.push_to_hub(new_model, use_temp_dir=False)


In [ ]:
#SFT训练环境：官方版:PyTorch/2.0.1/3.10(ubuntu22.04)/12.1.1

In [ ]:

###bitsandbytes相关报错 libbitsandbytes_cuda122.so不存在
##cuda 122 - not compatible with current bitsandbbytes #551
#https://github.com/TimDettmers/bitsandbytes/issues/551
#https://github.com/TimDettmers/bitsandbytes/issues
#bitsandbytes 0.40.0.post4 includes libbitsandbytes_cuda122.so so I believe this ticket should be closed.


#Ubuntu18.04+CUDA11安装bitsandbytes出现的问题
#https://blog.csdn.net/steptoward/article/details/135507131

In [ ]:
#怎样训练一个自己的大语言模型？：https://mp.weixin.qq.com/s/F60XYsT1dwUBmXN6r2G0eA
#Llama3-8B基于peft+trl进行SFT监督微调（Python代码模式）：https://blog.csdn.net/zhujiahui622/article/details/138196101